In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np 
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from google.colab import files
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import matplotlib.pyplot as plt
import math
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
random.seed(1234)

#Preprocessing 

In [ ]:
dd =  pd.read_csv("factors_2_numbers.csv")
#Delete empty columns
dd = dd.drop(['mg_sacu_seguim','ttm_seguim___31','ttm_seguim___32','ara2_seguim___3','islgt2_seguim___4','naco_seguim___5'],1)
dd = dd.iloc[:,1:]

In [ ]:
#Select categorical variables
categorical = ['sexe','data_visita_year','data_visita_month','data_visita_week','data_visita_day','procedencia','motiu_derivacio___1','motiu_derivacio___2','motiu_derivacio___3','motiu_derivacio___4','motiu_derivacio___5','motiu_derivacio___6','motiu_derivacio___7','motiu_derivacio___8','etiologia___1','etiologia___2','etiologia___3','etiologia___4','etiologia___5','etiologia___6','etiologia___7','etiologia___8','etiologia___9','etiologia___10','etiologia___11','etiologia___12','etiologia___13','antecedents___1','antecedents___2','antecedents___3','antecedents___4','antecedents___5','antecedents___6','antecedents___7','antecedents___8','antecedents___9','antecedents___10','antecedents___11','antecedents___12','antecedents___21','antecedents___13','antecedents___14','antecedents___15','antecedents___16','antecedents___17','antecedents___18','antecedents___20','neoplasia_estat','neoplasia_qt','tipus_iqprevia','insuf_mitral_seguim','ritme_base_seguim','trastorn_conduccio_t_v_1','marca','tto_ev_tipo___1','tto_ev_tipo___2','tto_ev_tipo___3','tto_ev_tipo___4','tto_ev_tipo___5','classe_funcional_seguim','ttm_seguim___1','ttm_seguim___2','ttm_seguim___3','ttm_seguim___4','ttm_seguim___5','ttm_seguim___6','ttm_seguim___7','ttm_seguim___8','ttm_seguim___9','ttm_seguim___10','ttm_seguim___11','ttm_seguim___12','ttm_seguim___13','ttm_seguim___14','ttm_seguim___15','ttm_seguim___16','ttm_seguim___17','ttm_seguim___18','ttm_seguim___19','ttm_seguim___20','ttm_seguim___21','ttm_seguim___22','ttm_seguim___23','ttm_seguim___24','ttm_seguim___25','ttm_seguim___26','ttm_seguim___27','ttm_seguim___28','ttm_seguim___29','ttm_seguim___30','ttm_seguim___33','diur_segui___1','diur_segui___2','diur_segui___3','mg_diur_segui','tiaz_seguim___1','tiaz_seguim___2','tiaz_seguim___3','mg_tiaz_seguim','antial_seguim___1','antial_seguim___2','antial_seguim___3','mg_anti_seguim','ieca_seguim___1','ieca_seguim___2','ieca_seguim___3','ieca_seguim___4','mg_ieca_seguim','ara2_seguim___1','ara2_seguim___2','ara2_seguim___4','mg_ara2_seguim','beta_seguim___1','beta_seguim___2','beta_seguim___3','beta_seguim___4','beta_seguim___5','mg_beta_seguim','islgt2_seguim___1','islgt2_seguim___2','islgt2_seguim___3','mg_islgt2_seguim','naco_seguim___1','naco_seguim___2','naco_seguim___3','naco_seguim___4','mg_naco_seguim','estacio_visita','IMC','mg_diur_segui']

One-hot encoding

In [ ]:
catdf = pd.get_dummies(dd[categorical].astype(str))
onehotdd = dd.copy()
onehotdd.drop(categorical, axis=1, inplace=True)
#One-hot encoding only on numerical variables
onehotdd = pd.concat([catdf, onehotdd], axis=1)

Scaled

In [ ]:
#Scaling numeric variables
numeric_cols = dd.drop(categorical, axis=1).columns
y = dd.iloc[:,dd.shape[1]-4:dd.shape[1]]
dd_scaled = dd.copy()
dd_scaled[numeric_cols]=(dd_scaled[numeric_cols]-dd_scaled[numeric_cols].mean())/dd_scaled[numeric_cols].std()
dd_scaled = dd_scaled.drop(y.columns, axis = 1)
dd_scaled = pd.concat([dd_scaled, y], axis=1)

###Final training data set 

In [ ]:
dd_train = dd.sample(frac=0.7,random_state=123)
dd_val = dd.drop(dd_train.index)

In order to compare the performance of the models with those trained in R with 3x3 CV, we use a 70% of the data to train the Sklearn models. We scale the data because in previous tests it has been seen that this preprocessing achieves the best performance for SVM.

In [ ]:
dd_train_scaled = dd_scaled.sample(frac=0.7,random_state=123) 
dd_val = dd.drop(dd_train_scaled.index)

#Training the models



In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

def model_function_completed(dd, model_name, balancing = "none", sampling ="standard",
                             LDA_priors = None, LDA_shrinkage=None, LDA_solver='svd',
                             QDA_priors = None, QDA_reg_param = -1,
                             SVM_kernel = 'duh', SVM_C = -1, SVM_CW = None, SVM_Gamma = "scale", SVM_Degree=3, SVM_Coef0=0.0, SVM_shrinking = False):

  oversample = RandomOverSampler(sampling_strategy='minority')
  undersample = RandomUnderSampler(sampling_strategy='majority')
  
  accuracy = [0,0,0,0]
  sensitivity = [0,0,0,0]
  specificity = [0,0,0,0]
  score = [0,0,0,0]

  #10 CV
  for j in range(10):

    n_cols = dd.shape[1]
    y = dd.iloc[:,n_cols-4:n_cols]
    
    #SAMPLING METHOD
    if (sampling == "standard"):
      
      train = dd.sample(frac=0.7,random_state=200+j) #random state is a seed value
      test = dd.drop(train.index)
      n_cols = dd.shape[1]
      x_train = train.iloc[:,:n_cols-4]
      y_train = train.iloc[:,n_cols-4:n_cols]
      x_test = test.iloc[:,:n_cols-4]
      y_test = test.iloc[:,n_cols-4:n_cols]
      
    
    elif (sampling == "balanced_resampling"):
      
      n_cols = dd.shape[1]
      x_train, x_test, y_train, y_test = train_test_split(dd.iloc[:,:n_cols-4],dd.iloc[:,n_cols-4:n_cols], test_size=0.3, random_state=123+j, stratify=y[['target_1', 'target_6', 'target_12']])

    #for each target
    for i in range(4):

      #BALANCING METHOD
      #oversample
      if (balancing == 'oversample'):
        x_train_balanced, y_train_balanced = oversample.fit_resample(x_train,
        y_train.iloc[:,i])
      #undersample
      elif (balancing == 'undersample'):
        x_train_balanced, y_train_balanced = undersample.fit_resample(x_train,
        y_train.iloc[:,i])
      #none
      elif (balancing == 'none'):
        x_train_balanced = x_train
        y_train_balanced = y_train.iloc[:,i]

      #MODEL NAME
      if (model_name == "LDA"):
        if (LDA_solver == 'svd'):
          #shrinkage is not supported for svd solver
          mod = LinearDiscriminantAnalysis(priors = LDA_priors, solver = LDA_solver)
        else:
          mod = LinearDiscriminantAnalysis(priors = LDA_priors, shrinkage = LDA_shrinkage, solver = LDA_solver)

      elif (model_name == "QDA"):
        mod = QuadraticDiscriminantAnalysis(priors = QDA_priors, reg_param = QDA_reg_param)

      elif (model_name == "SVM"):
        mod = SVC(kernel=SVM_kernel, max_iter = 200, C = SVM_C, class_weight = SVM_CW, gamma = SVM_Gamma, degree = SVM_Degree, coef0 = SVM_Coef0, shrinking = SVM_shrinking)
      
      mod.fit(x_train_balanced, y_train_balanced)
      test_prediction = mod.predict(x_test)
      cm = confusion_matrix(y_test.iloc[:,i], test_prediction)

      sensitivity[i] +=  (cm[1,1]/(cm[1,1]+cm[1,0]))/10
      specificity[i] +=  (cm[0,0]/(cm[0,0]+cm[0,1]))/10
      accuracy[i] += mod.score(x_test, y_test.iloc[:,i])/10



  
  
  for i in range (4):
    score[i] = 0.6*sensitivity[i] + 0.4*specificity[i]
    """results[i][0] = accuracy[i]
    results[i][1] = sensitivity[i]
    results[i][2] = specificity"""
    
  results = [accuracy,sensitivity,specificity,score]
  return results



##Hyperparameter optimization

In this section, the models are trained by optimizing their hyperparameters through tests with loops. The most efficient hyperameters (based in score) and their corresponding results in score (60% sensitivity + 40% specificity), accuracy, sensitivity and specificity are saved for each target.

###LDA

In [ ]:
prints = True
#First hyperparameter: balancing method
for method in ['none', 'oversample', 'undersample']: #undersample has the best performance for LDA
  if (prints): 
    print("With the following balancing method:", method)

  #Hyperparameter optimitzation:
  #Best score for each target, best hyperparameters for each target, other performance measurements for each target
  max = [[0,0,0,0],[0,0,0,0],[0,0,0,0]] 
  
  #Hyperparameter values
  solver = ['svd', 'lsqr', 'eigen']
  shrinkage = ['auto']#, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9]
  sampling = ["balanced_resampling", "standard"]

  #Second hyperparameter: solver (solv)
  for solv in solver:
    #Third hyperparameter: shrinkage (s)
    for s in shrinkage:
      #Fourth hyperparameter: priors (p)
      for p in range (1,20): 
        prior =  [0.05*p,1-p*0.05]
        #Fifth hyperparameter: sampling method (sa)
        for sa in sampling:
          #Training the model with all hyperparameter combinations
          results = model_function_completed(dd_train, "LDA", LDA_priors = prior, balancing = method, sampling = sa, LDA_shrinkage = s, LDA_solver = solv)
          #Select the hyperparameters that maximize the performance of the model for each target
          for j in range (4): 
            if (max[0][j] < results[3][j]): #Comparison based in score
              max[0][j] = results[3][j] #Maximum score
              max[1][j] = [solv, s, prior[0], sa] #Best solv, s, prior, sa based in score
              max[2][j] = [results[0][j], results[1][j], results[2][j]] #Accuracy, sensitivy, specificity

  print(max)

With the following balancing method: none
[[0.8380184331797234, 0.8093711551606289, 0.8064780763790664, 0.8080871072181844], [['lsqr', 'auto', 0.1, 'balanced_resampling'], ['lsqr', 'auto', 0.1, 'balanced_resampling'], ['lsqr', 'auto', 0.15000000000000002, 'balanced_resampling'], ['lsqr', 'auto', 0.15000000000000002, 'balanced_resampling']], [[0.8686098654708521, 0.8166666666666665, 0.8700460829493087], [0.7739910313901345, 0.8357142857142857, 0.7698564593301436], [0.7905829596412557, 0.8190476190476189, 0.7876237623762375], [0.7914798206278028, 0.8212121212121212, 0.7883995862272793]]]
With the following balancing method: oversample
[[0.765668202764977, 0.7389336978810663, 0.7749646393210747, 0.7848134984074099], [['lsqr', 'auto', 0.05, 'balanced_resampling'], ['lsqr', 'auto', 0.05, 'balanced_resampling'], ['lsqr', 'auto', 0.30000000000000004, 'balanced_resampling'], ['lsqr', 'auto', 0.45, 'balanced_resampling']], [[0.9313901345291481, 0.65, 0.9391705069124425], [0.8295964125560539, 0.

###QDA

In [ ]:
prints = True
#First hyperparameter: balancing method
for method in ['undersample']: #'none', 'oversample', 
  if (prints): 
    print("With the following balancing method:", method)

  #Hyperparameter optimitzation:
  #Best score for each target, best hyperparameters for each target, other performance measurements for each target
  max = [[0,0,0,0],[0,0,0,0],[0,0,0,0]] 
  
  #Hyperparameter values
  data = ['original', 'scaled']
  sampling = ["balanced_resampling", "standard"]

  #Second hyperparameter: data (d)
  for d in data:
    #Third hyperparameter: regularization parameter (rp)
    for rp in range (0,20): 
      reg_param = 0.05*rp
      #Fourth hyperparameter: priors (p)
      for p in range (1,20): 
        prior =  [0.05*p,1-p*0.05]
        #Fifth hyperparameter: sampling method (sa)
        for sa in sampling:
          #Training the model with all hyperparameter combinations
          if (d == 'original'):
            results = model_function_completed(dd_train, "QDA", balancing = method, sampling = sa, QDA_priors = prior, QDA_reg_param = reg_param)
          elif (d == 'scaled'):
            results = model_function_completed(dd_train_scaled, "QDA", balancing = method, sampling = sa, QDA_priors = prior, QDA_reg_param = reg_param)
          #Select the hyperparameters that maximize the performance of the model for each target
          for j in range (4): 
            if (max[0][j] < results[3][j]): #Comparison based in score
              max[0][j] = results[3][j] #Maximum score
              max[1][j] = [d, reg_param, prior[0], sa] #Best d, rp, prior, sa based in score
              max[2][j] = [results[0][j], results[1][j], results[2][j]] #Accuracy, sensitivy, specificity

  print(max)

With the following balancing method: undersample
[[0.6861751152073732, 0.6800546821599454, 0.7278925035360679, 0.7308924009936049], [['original', 0.6000000000000001, 0.15000000000000002, 'balanced_resampling'], ['original', 0.35000000000000003, 0.1, 'balanced_resampling'], ['original', 0.8500000000000001, 0.05, 'balanced_resampling'], ['original', 0.35000000000000003, 0.30000000000000004, 'standard']], [[0.6663677130044843, 0.7, 0.6654377880184331], [0.6340807174887892, 0.7142857142857144, 0.6287081339712918], [0.7511210762331839, 0.7095238095238096, 0.7554455445544553], [0.7744394618834082, 0.6950762327312078, 0.7846166533872005]]]


###SVM

####LINEAR

In [ ]:
prints = True
kernel = 'linear'
#First hyperparameter: balancing method
for method in ['none', 'oversample', 'undersample']: #undersampling has the best performance for SVM
  if (prints): 
    print("With the following balancing method:", method)

  #Hyperparameter optimitzation:
  #Best score for each target, best hyperparameters for each target, other performance measurements for each target
  max = [[0,0,0,0],[0,0,0,0],[0,0,0,0]] 
  
  #Hyperparameter values
  c_values = [1, 5, 10, 20, 50, 100]
  cw_values = ["balanced", None]
  sampling = ["balanced_resampling", "standard"]
  shrinking = [True, False]

  #Second hyperparameter: cost (c)
  for c in c_values: 
    #Third hyperparameter: class weight (cw)
    for cw in cw_values:
      #Fourth hyperparameter: shrinking (s)
      for s in shrinking:
        #Fifth hyperparameter: sampling method (sa)
        for sa in sampling:
          #Training the model with all hyperparameter combinations
          results = model_function_completed(dd_train_scaled, "SVM", SVM_kernel = kernel, balancing = method, sampling = sa, SVM_C = c, SVM_CW = cw, SVM_shrinking = s)
          #Select the hyperparameters that maximize the performance of the model for each target
          for j in range (4): 
            if (max[0][j] < results[3][j]): #Comparison based in score
              max[0][j] = results[3][j] #Maximum score
              max[1][j] = [c,cw,s,sa] #Best c, cw, s, s based in score
              max[2][j] = [results[0][j], results[1][j], results[2][j]] #Accuracy, sensitivy, specificity
  
  print(max)

With the following balancing method: none
[[0.6069604796932183, 0.6040855992413542, 0.6731036309913772, 0.6964365327790774], [[1, 'balanced', True, 'standard'], [1, 'balanced', True, 'standard'], [1, 'balanced', True, 'standard'], [1, 'balanced', True, 'balanced_resampling']], [[0.9591928251121076, 0.36000000000000004, 0.9774011992330457], [0.8860986547085202, 0.3906848706848707, 0.9241866920760794], [0.875336322869955, 0.5099968300197132, 0.917763832448873], [0.8834080717488789, 0.5482683982683982, 0.9186887345450961]]]
With the following balancing method: oversample
[[0.6069604796932183, 0.6040855992413542, 0.6731036309913772, 0.6964365327790774], [[1, 'balanced', True, 'standard'], [1, 'balanced', True, 'standard'], [1, 'balanced', True, 'standard'], [1, 'balanced', True, 'balanced_resampling']], [[0.9591928251121076, 0.36000000000000004, 0.9774011992330457], [0.8860986547085202, 0.3906848706848707, 0.9241866920760794], [0.875336322869955, 0.5099968300197132, 0.917763832448873], [0.

####POLY

In [ ]:
prints = True
kernel = 'poly'
#First hyperparameter: balancing method
for method in ['none', 'oversample', 'undersample']: #undersampling has the best performance for SVM
  if (prints): 
    print("With the following balancing method:", method)

  #Hyperparameter optimitzation:
  #Best score for each target, best hyperparameters for each target, other performance measurements for each target
  max = [[0,0,0,0],[0,0,0,0],[0,0,0,0]] 
  
  #Hyperparameter values
  degrees = [2, 3, 4, 6, 8, 10]
  coef = [0.0, 0.5,1.0, 5.0, 10.0, 20.0, 50.0, 100.0]
  c_values = [1, 5, 10, 20, 50, 100]
  cw_values = ["balanced", None]
  shrinking = [True, False]
  
  #Second hyperparameter: cost (c)
  for c in c_values: 
    #Third hyperparameter: class weight (cw)
    for cw in cw_values:
      #Fourth hyperparameter: shrinking (s)
      for s in shrinking:
        #Fifth hyperparameter: degree (d)
        for d in degrees:
          #Sixth hyperparameter: coeficient (co)
          for co in coef:
            #Training the model with all hyperparameter combinations
            results = model_function_completed(dd_train_scaled, "SVM", SVM_kernel=kernel, balancing = method, sampling = "balanced_resampling", SVM_C = c, SVM_CW = cw, SVM_shrinking = s, SVM_Coef0= co, SVM_Degree=d)
            #Select the hyperparameters that maximize the performance of the model for each target
            for j in range (4): 
              if (max[0][j] < results[3][j]): #Comparison based in score
                max[0][j] = results[3][j] #Maximum score
                max[1][j] = [c,cw,s,d,co] #Best c, cw, s, d, co based in score
                max[2][j] = [results[0][j], results[1][j], results[2][j]] #Accuracy, sensitivy, specificity
  
  print(max)

With the following balancing method: none


KeyboardInterrupt: ignored

####RBF

In [ ]:
prints = True
kernel = 'sigmoid'
#First hyperparameter: balancing method
for method in ['none', 'oversample', 'undersample']: #undersampling has the best performance for SVM
  if (prints): 
    print("With the following balancing method:", method)

  #Hyperparameter optimitzation:
  #Best score for each target, best hyperparameters for each target, other performance measurements for each target
  max = [[0,0,0,0],[0,0,0,0],[0,0,0,0]] 
  
  #Hyperparameter values
  gamma =  ['scale', 0.5, 1, 2, 5, 7, 10]
  c_values = [1, 3, 5, 10, 20, 50, 75, 100]
  cw_values = ["balanced", None]
  shrinking = [True, False]
  
  #Second hyperparameter: cost (c)
  for c in c_values: 
    #Third hyperparameter: class weight (cw)
    for cw in cw_values:
      #Fourth hyperparameter: shrinking (s)
      for s in shrinking:
        #Fifth hyperparameter: gamma (g)
        for g in gamma:
          #Training the model with all hyperparameter combinations
          results = model_function_completed(dd_train_scaled, "SVM", SVM_kernel=kernel, balancing = method, sampling = "balanced_resampling", SVM_C = c, SVM_CW = cw, SVM_shrinking = s, SVM_Gamma = g)
          #Select the hyperparameters that maximize the performance of the model for each target
          for j in range (4): 
            if (max[0][j] < results[3][j]): #Comparison based in score
              max[0][j] = results[3][j] #Maximum score
              max[1][j] = [c,cw,s,g] #Best c, cw, s, g based in score
              max[2][j] = [results[0][j], results[1][j], results[2][j]] #Accuracy, sensitivy, specificity

  print(max)

With the following balancing method: none
[[0.5999999999999999, 0.5999999999999999, 0.5999999999999999, 0.5999999999999999], [[1, 'balanced', True, 'scale'], [1, 'balanced', True, 'scale'], [1, 'balanced', True, 'scale'], [1, 'balanced', True, 'scale']], [[0.026905829596412557, 0.9999999999999999, 0.0], [0.06278026905829596, 0.9999999999999999, 0.0], [0.09417040358744395, 0.9999999999999999, 0.0], [0.09506726457399103, 0.9999999999999999, 0.0]]]
With the following balancing method: oversample
[[0.818294930875576, 0.7569924812030076, 0.7492786421499292, 0.7568837694003943], [[3, None, False, 'scale'], [5, 'balanced', True, 'scale'], [50, 'balanced', False, 'scale'], [50, 'balanced', False, 'scale']], [[0.6295964125560538, 0.9500000000000001, 0.62073732718894], [0.5457399103139013, 0.9142857142857144, 0.5210526315789473], [0.5466367713004484, 0.9095238095238096, 0.5089108910891088], [0.5511210762331837, 0.9199134199134199, 0.5123392936308556]]]
With the following balancing method: unders